In [1]:
include("JQlaw.jl")
using .JQlaw
using Evolutionary
# using Plotly

## Simple Orbit Raise Example

In [2]:
orbit_inital = KeplarianOrbit(
                450000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

orbit_target = KeplarianOrbit(
                600000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

Q_Params = QParams(
                1,
                1,
                R_e+300000., 
                1,
                0,
                0,
                0,
                0,
                3,
                4,
                2,
                10^-10,
                0.5,
                0,
                10, 
                500.,
                10^6,
                10^3)

Sat_Params = sat_params(100, 0.8, 0.5/100., 3100)

time_initial = 59715.0 # MJD

Q_hist, orbit_hist, time_hist, mass_hist = Qlaw(orbit_inital, orbit_target, Q_Params, Sat_Params, time_initial)

print("Spent Prop: ", mass_hist[1] - mass_hist[end], "[kg]\n")
print("Transfer Time: ", time_hist[end] - time_hist[1], "[days]\n")

# # change effectivity ratio to be more propellant safe...

# Q_Params = QParams(
#                 1,
#                 1,
#                 R_e+300000., 
#                 1,
#                 0,
#                 0,
#                 0,
#                 0,
#                 3,
#                 4,
#                 2,
#                 10^-10,
#                 0.9,
#                 0,
#                 10, 
#                 500.,
#                 10^6,
#                 10^3)

# Q_hist, orbit_hist, time_hist, mass_hist = Qlaw(orbit_inital, orbit_target, Q_Params, Sat_Params, time_initial)

# print("Spent Prop: ", mass_hist[1] - mass_hist[end], "[kg]\n")
# print("Transfer Time: ", time_hist[end] - time_hist[1], "[days]\n")

Spent Prop: 0.36119577687757953[kg]
Transfer Time: 0.896990740657202[days]


In [3]:
function QlawArray(x)
    q_params = QParams(
        1,
        1,
        R_e+300000., 
        1,
        0,
        0,
        0,
        0,
        a[1],
        a[2],
        a[3],
        10^-10,
        a[4],
        a[5],
        a[6],
        a[7],
        10^6,
        10^3
    )
    Q_hist, orbit_hist, time_hist, mass_hist = Qlaw(orbit_inital, orbit_target, Q_Params, Sat_Params, time_initial) 
    return mass_hist[1] - mass_hist[end]
end
    

18-element Vector{Float64}:
    1.0
    1.0
    6.671e6
    1.0
    0.0
    0.0
    0.0
    0.0
    3.0
    4.0
    2.0
    1.0000000000000006e-10
    0.5
    0.0
   10.0
  500.0
    1.0e6
 1000.0

In [4]:
initial_params = [3, 4, 2, 0.5, 0., 10., 500.]

7-element Vector{Float64}:
   3.0
   4.0
   2.0
   0.5
   0.0
  10.0
 500.0

In [23]:
res = Evolutionary.optimize(x->-QlawArray(x),
    initial_params,
    GA())


 * Status: success

 * Candidate solution
    Minimizer:  [3.0, 4.0, 2.0,  ...]
    Minimum:    -0.36119577687757953
    Iterations: 12

 * Found with
    Algorithm: GA[P=50,x=0.8,μ=0.1,ɛ=0]

 * Convergence measures
    |f(x) - f(x')| = 0.0 ≤ 1.0e-12

 * Work counters
    Seconds run:   8.7435 (vs limit Inf)
    Iterations:    12
    f(x) calls:    650
